In [9]:
library(lme4)
suppressPackageStartupMessages(library(dplyr))
suppressPackageStartupMessages(library(tidyr))
suppressPackageStartupMessages(library(tibble))
suppressPackageStartupMessages(library(plyr))
suppressPackageStartupMessages(library(stringr))
suppressPackageStartupMessages(library(parallel))

In [1]:
Olink<-read.csv("/home//jupyter/BRI_Figures_Final_V2/Figure2/01_Olink_Analysis/Olink_BRI.csv")

In [4]:
meta<-read.csv('/home/jupyter/BRI_Figures_Final_V2/Dataset/scRNA_meta_data-2024-05-09.csv')
meta_sub<-meta %>% filter(sample.visitName=='Flu Year 2 Day 0')
dim(meta_sub)

[1] 84 32

In [5]:
rownames(meta)<-meta$sample.sampleKitGuid

In [6]:
Olink<-Olink %>% filter(sample.sampleKitGuid %in% unique(meta$sample.sampleKitGuid))

In [7]:
Olink<-left_join(Olink,meta, by = c("sample.sampleKitGuid"))

In [13]:
Olink_sub<-Olink %>%filter(sample.visitName == 'Flu Year 2 Day 0')

In [21]:
# remove these protein since there are only 6 vaild samples samples 
olink_to_remove<-as.data.frame(table(Olink_sub$Assay)) %>% filter(Freq==6) %>% select(Var1)%>% pull()

In [22]:
Olink<-Olink %>% filter(!Assay %in% olink_to_remove)

In [23]:
res_list <- mclapply(unique(Olink$Assay), function(x) {
  Olink_sub <- Olink %>%filter(sample.visitName == 'Flu Year 2 Day 0', Assay == x)
  Olink_sub$NPX_bridged <- as.numeric(Olink_sub$NPX_bridged)
  Olink_sub <- Olink_sub[!is.na(Olink_sub$NPX_bridged), ]
  
  res <- lm(NPX_bridged ~ cohort.cohortGuid + CMV + subject.biologicalSex, data = Olink_sub)
  res <- as.data.frame(summary(res)$coefficients)[2:4, ]
  res$factor <- c('Age', 'CMV', 'Sex')
  res$Assay <- x

  res$Log2FC_Age <- Olink_sub %>%
    dplyr::group_by(Assay, cohort.cohortGuid) %>%
    dplyr::summarise(Mean_Expression = mean(NPX_bridged, na.rm = TRUE)) %>%
    spread(cohort.cohortGuid, Mean_Expression) %>%
    mutate(Log2FC = log2(BR2 / BR1)) %>%
    ungroup() %>%
    select(Log2FC) %>%
    pull()
  
  res$delta_changes_Age <- Olink_sub %>%
    dplyr::group_by(Assay, cohort.cohortGuid) %>%
    dplyr::summarise(Mean_Expression = mean(NPX_bridged, na.rm = TRUE)) %>%
    spread(cohort.cohortGuid, Mean_Expression) %>%
    mutate(diff = BR2 - BR1) %>%
    select(diff) %>%
    pull()

  res$Log2FC_CMV <- Olink_sub %>%
    dplyr::group_by(Assay, CMV) %>%
    dplyr::summarise(Mean_Expression = mean(NPX_bridged, na.rm = TRUE)) %>%
    spread(CMV, Mean_Expression) %>%
    mutate(Log2FC = log2(Positive / Negative)) %>%
    ungroup() %>%
    select(Log2FC) %>%
    pull()
  
  res$delta_changes_CMV <- Olink_sub %>%
    dplyr::group_by(Assay, CMV) %>%
    dplyr::summarise(Mean_Expression = mean(NPX_bridged, na.rm = TRUE)) %>%
    spread(CMV, Mean_Expression) %>%
    mutate(diff = Positive - Negative) %>%
    select(diff) %>%
    pull()

  res$Log2FC_Sex <- Olink_sub %>%
    dplyr::group_by(Assay, subject.biologicalSex) %>%
    dplyr::summarise(Mean_Expression = mean(NPX_bridged, na.rm = TRUE)) %>%
    spread(subject.biologicalSex, Mean_Expression) %>%
    mutate(Log2FC = log2(Female / Male)) %>%
    ungroup() %>%
    select(Log2FC) %>%
    pull()
  
  res$delta_changes_Sex <- Olink_sub %>%
    dplyr::group_by(Assay, subject.biologicalSex) %>%
    dplyr::summarise(Mean_Expression = mean(NPX_bridged, na.rm = TRUE)) %>%
    spread(subject.biologicalSex, Mean_Expression) %>%
    mutate(diff = Female - Male) %>%
    select(diff) %>%
    pull()

  return(res)
}, mc.cores = 60)

In [24]:
res_all<-do.call(rbind,res_list)
res_all<-res_all %>%group_by(factor) %>% mutate(padj=p.adjust(`Pr(>|t|)`))
write.csv(res_all,"Olink_lm_Result.csv")